# Régime d'oscillations forcées par superposition modale (mouvement d'ensemble)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as la
from scipy.integrate import odeint
import scipy.fftpack
from scipy import signal
plt.rcParams['figure.figsize'] = [12, 3]
from scipy import optimize
import functools
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

## Paramètres de la poutre

In [ ]:
n_elem=10; # nombre d'éléments --> A MODIFIER
E=203000000000; # module d'Young --> Acier inoxydable
rho=7900; # masse volumique --> Acier inoxydable
e=0.0005; # épaisseur de la lame --> Réglet SAM  781-3
b=0.013; # largeur de la lame --> Réglet SAM  781-3
l=0.265; # longueur de la lame --> Réglet SAM  781-3
ksi_rayleigh=0*0.0059; # taux d'amortissement (visqueux) de la lame --> A MODIFIER expérimental, oscillation libre
S=e*b; # Section de la poutre
x=np.linspace(0,l,101); # Coordonnées le long de la poutre
m = rho*S*l #masse totale

## Paramètres d'intégration

In [ ]:
deltat=0.02; # pas de temps --> A MODIFIER
duree=7; # plage d'integration --> A MODIFIER
nb_pt=int(duree/deltat); # Discrétisation temporelle
qmax=0.013; # entraxe axe moteur / goupille manivelle
fmax=100000; 
omega_force=1000*2*np.pi;

delta_om=1; # --> A MODIFIER
om_min=0.001; # --> A MODIFIER
om_max=3000; # --> A MODIFIER

## Construction des matrices de masse et de raideur

\begin{equation}
\left[ M \right] =
\frac{\rho S l}{420} \left[ \begin{array}{c c}
156 & 22l & 54 & -13l \\
22l & 4l^2 & 13l & -3l^2 \\
54 & 13l & 156 & -22l \\
-13l & -3l^2 & -22l & 4l^2 
\end{array} \right]
\end{equation}

\begin{equation}
\left[ K \right] =
\frac{EI}{l^3} \left[ \begin{array}{c c}
12 & 6l & -12 & 6l \\
6l & 4l^2 & -6l & 2l^2 \\
-12 & -6l & 12 & -6l \\
6l & 2l^2 & -6l & 4l^2
\end{array} \right]
\end{equation}


In [ ]:
Iz = b*e**3/12
l_elem=l/n_elem; # longueur d'un élément

k_elem=E*Iz/(l_elem**3)*np.array([[12,6*l_elem,-12,6*l_elem],[6*l_elem,4*l_elem**2,-6*l_elem,2*l_elem**2],[-12,-6*l_elem,12,-6*l_elem],[6*l_elem,2*l_elem**2,-6*l_elem,4*l_elem**2]]);
m_elem=rho*S*l_elem/420*np.array([[156,22*l_elem,54,-13*l_elem],[22*l_elem,4*l_elem**2,13*l_elem,-3*l_elem**2],[54,13*l_elem,156,-22*l_elem],[-13*l_elem,-3*l_elem**2,-22*l_elem,4*l_elem**2]]);


k_ass0=np.zeros(((n_elem+1)*2,(n_elem+1)*2));
m_ass0=np.zeros(((n_elem+1)*2,(n_elem+1)*2));

#print("k_ass0 =",k_ass0);
#print("m_ass0 =",m_ass0);

for j in range(n_elem):
    k_ass0[j*2:j*2+4,j*2:j*2+4]=k_ass0[j*2:j*2+4,j*2:j*2+4]+k_elem;
    m_ass0[j*2:j*2+4,j*2:j*2+4]=m_ass0[j*2:j*2+4,j*2:j*2+4]+m_elem;

k_ass=k_ass0[2:2*(n_elem+1),2:2*(n_elem+1)];
m_ass=m_ass0[2:2*(n_elem+1),2:2*(n_elem+1)];

#m_ass[len(m_ass[0,:])-1,len(m_ass[:,0])-1] += 0.01 #accéléromètre

print("k_ass =\n",k_ass);
print("m_ass =\n",m_ass);

## Pulsations et modes analytiques

$k \in \mathbb{N}^{*}$,
\begin{equation}
\omega_{k} = ((2k-1) \frac{\pi}{2})^2  \sqrt{\frac{EI_z}{ml^3}}
\end{equation}

\begin{equation}
\phi_{k}(x)= cf \ page \ 45
\end{equation}

In [ ]:
omega_ana=np.zeros((n_elem,1));
Phi_ana=np.zeros((n_elem,len(x)));
for k in range(n_elem):
    if k==0:
        omega_ana[k] = (1.87**2)*np.sqrt(E*Iz/(m*l**3))
    else:
        omega_ana[k]=(((2*(k+1)-1)*np.pi/2)**2)*np.sqrt(E*Iz/(m*l**3));
    for j in range(len(x)):
        A = (((omega_ana[k]**2)*rho*S/E/Iz)**(1/4))*x[j]
        B = (((omega_ana[k]**2)*rho*S/E/Iz)**(1/4))*l
        Phi_ana[k,j]=np.cos(A)-np.cosh(A)+(np.sin(B)-np.sinh(B))/(np.cos(B)+np.cosh(B))*(np.sin(A)-np.sinh(A));
        
print("omega_ana = ", omega_ana)



#print("Phi_ana = ", Phi_ana)
print("freq = ",omega_ana/2/np.pi)

## Résolution du problème aux valeurs propres

La détermination des pulsations propres et des modes propres consiste à résoudre le problème suivant :

\begin{equation}
det \left( \left[ M \right]^{-1} \left[ K \right] - \omega^{2}_{i} \left[ I \right] \right) = 0
\end{equation}

In [ ]:
val_propres, vect_propres = la.eig(np.dot(la.inv(m_ass),k_ass)); # problème aux valeurs propres
omegar=np.sqrt(np.real(val_propres));
print("omegar =",omegar);
o_omegar=np.argsort(omegar); # tri des modes (dans le sens croissant)
print("o_omegar =",o_omegar);
print("omegar[o_omegar] =",omegar[o_omegar]);

#On sépare les vecteurs propres correspondants aux déplacements tranversaux des déplacements angulaires:
#Conditions aux limites
vps_T = np.zeros((1,2*n_elem)) #Transversaux
vps_A = np.zeros((1,2*n_elem)) #Angulaire
#print(vps_T)
for i in range(n_elem):
    vps_T = np.concatenate((vps_T,[vect_propres[i*2,:]]),axis=0)
    vps_A = np.concatenate((vps_A,[vect_propres[i*2+1,:]]),axis=0)

vps=np.concatenate((np.zeros((1,2*n_elem)),vect_propres), axis=0); # ajout des conditions aux limites
#print("vps =",vps);
print("vps[o_omegar] =",np.real(vps[:,o_omegar]));
print("vps_T= ",vps_T)

In [ ]:
plt.figure();
plt.plot(range(1,n_elem+1),omega_ana,'-o');
plt.plot(range(1,2*n_elem+1),omegar[o_omegar],'--o');
plt.xlabel("n");
plt.ylabel("$\omega_{n}$");
#plt.xlim(0.75,1.25)
#plt.ylim(0,100)
plt.figure();

#for i in range(n_elem):
for i in range(3):    #3 premiers modes
    plt.plot(x,Phi_ana[i,:]/np.max(np.abs(Phi_ana[i,:])),'-');
#print(np.arange(0,2*n_elem-1,1)/(n_elem)*0.265)
#for i in range(2*n_elem):
for i in range(3):
    plt.plot(np.arange(0,n_elem+1,1)/(n_elem)*0.265,-np.real(vps_T[:,o_omegar][:,i])/np.max(np.abs(np.real(vps_T[:,o_omegar][:,i])))*np.sign(np.real(vps_T[:,o_omegar][1,i])),'--o');

plt.xlabel("n");
plt.ylabel("$\Phi_{n}$");

## Application des conditions initiales

Le vecteur $\left\lbrace q \right\rbrace$ s'écrit comme la combinaison linéaire des $N$ vecteurs propres $\left\lbrace x_{i=1..N} \right\rbrace$ :

\begin{equation}
\left\lbrace q \right\rbrace = \Sigma^{N}_{i=1} \eta_{i} \left\lbrace x_{i} \right\rbrace
\end{equation}

In [ ]:
qq=np.zeros((2*n_elem,nb_pt));
eta=np.zeros((2*n_elem,nb_pt));
for i in range(n_elem):
    qq[i,0]=0 #(i+1)*qmax*l_elem; # déformation homogène de la poutre à t=0
print("shape(qq) = ",np.shape(qq))
print("shape(vps[1:2*n_elem+1]) = ",np.shape(vps[1:2*n_elem+1]))
eta[:,0]=np.dot(np.linalg.inv(vps[1:2*n_elem+1,o_omegar]),qq[:,0]); # détermination des composantes initiales dans la base modale à partir de celles dans la base physique
print(eta[:,0]);
print(np.shape(eta))

## Affectation des paramètres et des efforts extérieurs

In [ ]:
ksi=np.zeros((2*n_elem,1));
omegar_red=np.zeros((2*n_elem,1));
dot_eta_init=np.zeros((2*n_elem,1));
gg=np.zeros((2*n_elem,nb_pt));
t=np.linspace(0,duree,nb_pt);
for i in range(2*n_elem):
    ksi[i]=ksi_rayleigh; # taux d'amortissement
    omegar_red[i]=omegar[o_omegar][i]*np.sqrt(1-ksi[i]**2); # pulsation réduite
    dot_eta_init[i]=0; # vitesses initiales nulles
    for pt in range(nb_pt):
        #gg[n_elem-1,pt]=1000*np.sin(2*np.pi*4*t[pt]); # force sinusoïdale

        gg[i,pt]=rho*l*S/n_elem*qmax*np.sin(0.5*omegar[o_omegar][0]*t[pt])*(0.5*omegar[o_omegar][0])**2;
        if i==n_elem-1:
            gg[i,pt]=1/2*rho*l*S/n_elem*qmax*np.sin(0.5*omegar[o_omegar][0]*t[pt])*(0.5*omegar[o_omegar][0])**2;

plt.plot(t,gg[n_elem-1,:],'-o')
print(np.shape(gg))
plt.xlabel("t");
plt.ylabel("$g(t)$");
print(gg)

## Intégration numérique (transitoire)

L'équation d'équilibre est donnée par le système matriciel suivant :

\begin{equation}
\label{support}
\begin{array}{r}
\left[ \begin{array}{c c}
\left[ M_{f \, f} \right] & \left[ M_{f \, \overline{f}} \right] \\
\\
\left[ M_{\overline{f} \, f} \right] & \left[ M_{\overline{f} \, \overline{f}} \right]
\end{array} \right]
\left\lbrace
\begin{array}{c}
\left\lbrace \ddot{q}_{f} \right\rbrace \\
\\
\left\lbrace \ddot{q}_{\overline{f}} \right\rbrace
\end{array}
\right\rbrace
+
\left[ \begin{array}{c c}
\left[ C_{f \, f} \right] & \left[ C_{f \, \overline{f}} \right] \\
\\
\left[ C_{\overline{f} \, f} \right] & \left[ C_{\overline{f} \, \overline{f}} \right]
\end{array} \right]
\left\lbrace
\begin{array}{c}
\left\lbrace \dot{q}_{f} \right\rbrace \\
\\
\left\lbrace \dot{q}_{\overline{f}} \right\rbrace
\end{array}
\right\rbrace
+
\left[ \begin{array}{c c}
\left[ K_{f \, f} \right] & \left[ K_{f \, \overline{f}} \right] \\
\\
\left[ K_{\overline{f} \, f} \right] & \left[ K_{\overline{f} \, \overline{f}} \right]
\end{array} \right]
\left\lbrace
\begin{array}{c}
\left\lbrace q_{f} \right\rbrace \\
\\
\left\lbrace q_{\overline{f}} \right\rbrace
\end{array}
\right\rbrace
\\
\\
=
\left\lbrace
\begin{array}{c}
\left\lbrace g_{f}(t) \right\rbrace \\
\\
\left\lbrace g_{\overline{f}}(t) \right\rbrace
\end{array}
\right\rbrace
\end{array}
\end{equation}

Où $g_{f}$ représente les efforts de réaction du support tandis que $g_{\overline{f}}$ représente les efforts extérieurs s'exerçant sur la structure.

Le vecteur des accélérations nodales se décompose en une composante "d'entraînement" et une composante "relative" :

\begin{equation}
\begin{array}{l c c c c}
\left\lbrace \ddot{q} \right\rbrace = \left\lbrace \begin{array}{c} \ddot{q}_{f} \\ \ddot{q}_{\overline{f}} \end{array} \right\rbrace & = & \underbrace{\left\lbrace \begin{array}{c} \left\lbrace u_{f} \right\rbrace \ddot{\Phi} \\ \\ \left\lbrace u_{\overline{f}} \right\rbrace \ddot{\Phi} \end{array} \right\rbrace} & + & \underbrace{\left\lbrace \begin{array}{c} \left\lbrace 0 \right\rbrace \\ \\ \left\lbrace \ddot{q}_{r} \right\rbrace \end{array} \right\rbrace}
\\
& & accélération &  & accélération \\
& & d'entraînement &  & relative \\
\end{array}
\end{equation}

On cherche à résoudre l'équation suivante :

\begin{equation}
\left[ M_{\overline{f} \, \overline{f}} \right] \left\lbrace \ddot{q}_{r} \right\rbrace + \left[ C_{\overline{f} \, \overline{f}} \right] \left\lbrace \dot{q}_{r} \right\rbrace + \left[ K_{\overline{f} \, \overline{f}} \right] \left\lbrace q_{r} \right\rbrace = \left\lbrace g_{\overline{f}}(t) \right\rbrace - \left( \left[ M_{\overline{f} \, f} \right] \left\lbrace u_{f} \right\rbrace + \left[ M_{\overline{f} \, \overline{f}} \right] \left\lbrace u_{\overline{f}} \right\rbrace \right) \ddot{\Phi}
\end{equation}

Le changement de base permet d'obtenir pour chaque mode i ($1\le i \le n_{modes}$) :

\begin{equation}
\lambda_{i} \eta_{i} + 2 \xi_{i} \omega_{i} \dot{\eta_{i}} + \ddot{\eta_{i}} = \frac{\left\lbrace x_{i} \right\rbrace^{T} \left\lbrace F \right\rbrace}{\mu_{i}} = \frac{\left\lbrace x_{i} \right\rbrace^{T} \left\lbrace g(t) \right\rbrace}{\mu_{i}}
\end{equation}

Soit le mode $i$, la solution de l'équation différentielle associée est donnée par :

\begin{equation}
\begin{array}{l c l}
\eta_{i}(t) & = & e^{-\xi_{i}\omega_{i}t} \left[ cos \left(\omega_{i}\sqrt{1-\xi_{i}^{2}}t \right)+\frac{\xi_{i}}{\sqrt{1-\xi_{i}^{2}}} sin \left(\omega_{i}\sqrt{1-\xi_{i}^{2}}t \right) \right]\eta_{i}(0) + e^{-\xi_{i}\omega_{i}t}\left[ \frac{sin \left(\omega_{i}\sqrt{1-\xi_{i}^{2}}t \right)}{\omega_{i}\sqrt{1-\varepsilon_{i}^{2}}} \right]\dot{\eta_{i}}(0) \\ & & \\ & &
+ \int_{0}^{t} e^{-\xi_{i}\omega_{i}(t-\tau)} \frac{sin \left(\omega_{i}\sqrt{1-\xi_{i}^{2}}(t-\tau) \right)}{\omega_{i}\sqrt{1-\xi_{i}^{2}}} \frac{\left\lbrace x_{i} \right\rbrace^{T} \left\lbrace g(\tau) \right\rbrace }{\mu_{i}}d\tau
\end{array}
\end{equation}

In [ ]:
mu=np.zeros((2*n_elem,1));
TT=np.zeros((2*n_elem,nb_pt));

for i in range(n_elem):
    print("i =",i);
    mu_temp=np.dot(m_ass,vps[1:2*n_elem+1,o_omegar][:,i]);
    mu[i]=np.vdot(vps[1:2*n_elem+1,o_omegar][:,i],mu_temp);
    for pt in range(nb_pt):
        #print("pt =",pt);
        delta_tau=deltat;
        tau1=0;
        TT[i,pt]=0;
        for k in range(pt): # intégration numérique liée au second membre par la méthode des trapèzes
            f_tau1=np.exp(-ksi[i]*omegar[o_omegar][i]*(t[pt]-tau1))*np.sin(omegar_red[i]*(t[pt]-tau1))/omegar_red[i]*np.dot(vps[1:2*n_elem+1,o_omegar][:,i],gg[:,k])/mu[i];
            f_tau2=np.exp(-ksi[i]*omegar[o_omegar][i]*(t[pt]-(tau1+delta_tau)))*np.sin(omegar_red[i]*(t[pt]-(tau1+delta_tau)))/omegar_red[i]*np.dot(vps[1:2*n_elem+1,o_omegar][:,i],gg[:,k+1])/mu[i];
            TT[i,pt]=TT[i,pt]+delta_tau*(f_tau1+f_tau2)/2;
            tau1=tau1+delta_tau;
        eta[i,pt]=np.exp(-ksi[i]*omegar[o_omegar][i]*t[pt])*(np.cos(omegar_red[i]*t[pt])+ksi[i]/np.sqrt(1-ksi[i]**2)*np.sin(omegar_red[i]*t[pt]))*eta[i,0]+np.exp(-ksi[i]*omegar[o_omegar][i]*t[pt])/omegar_red[i]*np.sin(omegar_red[i]*t[pt])*dot_eta_init[i]+TT[i,pt];

## Calcul des paramètres généralisés

Retour à la base physique de départ :

\begin{equation}
\left\lbrace q \right\rbrace = \Sigma^{N}_{i=1} \eta_{i} \left\lbrace x_{i} \right\rbrace
\end{equation}

In [ ]:
for pt in range(1,nb_pt):
    qq[:,pt]=np.dot(vps[1:2*n_elem+1,o_omegar],eta[:,pt]);
print(np.shape(vps))
print(np.shape(eta))
qf=np.zeros((2*n_elem+1,nb_pt));
qf[1:2*n_elem+1,:]=qq;

In [ ]:
plt.figure();
for i in range(3):
    plt.plot(t,eta[2*i,:],'-');
    print(2*i)
plt.xlabel("$t$");
plt.ylabel("$\eta$");

plt.figure();
for i in range(n_elem):
    plt.plot(t,qf[2*i,:],'-');
plt.xlabel("$t$");
plt.ylabel("$q$");

## Matrice d'amortissement

In [ ]:
if n_elem==1:
    alphaa=ksi_rayleigh*2/omegar[o_omegar][0];
    betaa=0*ksi_rayleigh*2*omegar[o_omegar][0];
else:
    alphaa=ksi_rayleigh*2/(omegar[o_omegar][0]+omegar[o_omegar][2]);
    betaa=ksi_rayleigh*2*omegar[o_omegar][0]*omegar[o_omegar][2]/(omegar[o_omegar][0]+omegar[o_omegar][2]);
    
c_ass0=alphaa*k_ass0+betaa*m_ass0;
c_ass=c_ass0[1:2*n_elem+1,1:2*n_elem+1];
#print("c_ass",c_ass);

## Intégration numérique (harmonique)

On recherche une solution sous la forme suivante :

\begin{equation}
\left\lbrace q \right\rbrace = \left\lbrace q(x) \right\rbrace e^{j \omega t}
\end{equation}

D'où l'équation d'équilibre dynamique suivante :

\begin{equation}
-\omega^{2} \left[ M \right] \left\lbrace q \right\rbrace+j\omega \, \left[ C \right] \left\lbrace q \right\rbrace + \left[ K \right] \left\lbrace q \right\rbrace = \left[ S(\omega) \right] \left\lbrace q \right\rbrace = \left\lbrace a(\omega) \right\rbrace
\end{equation}

On cherche à déterminer $\left\lbrace b_{k} \right\rbrace$ pour chaque pulsation d'excitation $\omega_{k}$, avec :

\begin{equation}
\left[ S(\omega) \right] \left\lbrace b \right\rbrace = \left[ X \right]^{T} \left\lbrace a(\omega) \right\rbrace
\end{equation}

In [ ]:
om=np.arange(om_min,om_max,delta_om);
nb_pom=len(om);
qw=np.zeros((2*n_elem,nb_pom),dtype=complex);
for pom in range(nb_pom):
    print("pom =",pom,om[pom]);
    aa=rho*l*S/n_elem*qmax*np.ones((2*n_elem,1))*(om[pom])**2;
    aa[2*n_elem-1]=1/2*rho*l*S/n_elem*qmax*(om[pom])**2;
    SS=k_ass-(om[pom]**2)*m_ass+1j*om[pom]*c_ass;
    invS=la.inv(SS);
    vpsa=np.dot(np.transpose(vps[1:2*n_elem+1,o_omegar]),aa);
    bb=np.dot(invS,vpsa);
    qw[:,pom:pom+1]=np.dot(vps[1:2*n_elem+1,o_omegar],bb);

In [ ]:
for i in range(6):
    print("i =",i,omegar[o_omegar][i])
    plt.plot(om,np.abs(np.real(qw[2,:])),'-');
    plt.plot(omegar[o_omegar][i]*np.ones((2,1)),[0,1.2*np.max(np.abs(qw))],'--')
plt.xlabel("$\omega$");
plt.ylabel("$q$");
plt.xlim(0,1000)
plt.ylim(0,1)